In [8]:
import tensorflow as tf
from tensorflow import keras, sparse 
from keras import layers, applications
# from keras.preprocessing import image
from keras.applications.efficientnet import EfficientNetB0, EfficientNetB5, EfficientNetB7
# from keras.applications.resnet50 import ResNet50
from keras.applications.densenet import DenseNet121
# from keras.applications import EfficientNetB0
from keras.applications.efficientnet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator


import os 
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential

# Data preprocessing 
from keras.preprocessing.image import load_img, img_to_array
 

In [9]:
def getDataset(dataFolder, subset, imageSize = (224, 224), batchSize = 32):
    train_ds = keras.utils.image_dataset_from_directory(
      dataFolder,
      seed=123,
      image_size=imageSize,
      batch_size=batchSize)
    return train_ds


def configurePerformance(train_ds, val_ds): 
    AUTOTUNE = tf.data.AUTOTUNE
    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
    return train_ds, val_ds

# def buildModel(dropoutRate, numClasses, inpShape = (224, 224, 3)):
import pickle

def dumpModel(modelName, phase): 
    # Save the trained model as a pickle string.
    modelName = "model_" + modelName + "_ " + phase + ".pkl"
    pickle.dump(model, open(modelName, 'wb'))




In [10]:
EPOCHS = 5
modelName = "ResNet50"
inpShape =  (224, 224, 3)
trainingFolder = 'data/x5/train/RGB/'
testingFolder = 'data/x5/test_with_labels/RGB/'
train_ds = getDataset(trainingFolder, "training")
val_ds =  getDataset(testingFolder, "validation")

dropoutRate = 0.2
numClasses = 7 
inp = layers.Input(shape=inpShape)
baseModel = tf.keras.applications.resnet50.ResNet50(weights="imagenet",
                   include_top = False) 

baseModel.trainable = False 
x = baseModel(inp, training=False)
x =  layers.GlobalAveragePooling2D(name="avg_pool")(x)
x = layers.Dropout(dropoutRate, noise_shape=None, seed=None)(x)
out = layers.Dense(numClasses,activation="softmax", name = "pred")(x)
model = keras.Model(inp, out, name="FeatureExtraction-ResNet50")
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
          loss=tf.keras.losses.SparseCategoricalCrossentropy(),
          metrics=['accuracy'])

# dumpModel(modelName, "phase1")

Found 6000 files belonging to 7 classes.
Found 1500 files belonging to 7 classes.
94781440/94765736 [==============================] - 7s 0us/step


In [11]:
# Feature extraction without the top layers 
hist_results = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=EPOCHS
)


Epoch 1/5


2022-03-11 14:24:18.478149: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


188/188 [==============================] - ETA: 0s - loss: 0.8308 - accuracy: 0.8270

2022-03-11 14:25:31.685290: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


188/188 [==============================] - 92s 467ms/step - loss: 0.8308 - accuracy: 0.8270 - val_loss: 1.6082 - val_accuracy: 0.6773
Epoch 2/5
188/188 [==============================] - 79s 421ms/step - loss: 0.3811 - accuracy: 0.9035 - val_loss: 2.6691 - val_accuracy: 0.6413
Epoch 3/5
188/188 [==============================] - 78s 414ms/step - loss: 0.3937 - accuracy: 0.9122 - val_loss: 2.9186 - val_accuracy: 0.6567
Epoch 4/5
188/188 [==============================] - 79s 417ms/step - loss: 0.3173 - accuracy: 0.9305 - val_loss: 1.4877 - val_accuracy: 0.7467
Epoch 5/5
188/188 [==============================] - 77s 409ms/step - loss: 0.3053 - accuracy: 0.9362 - val_loss: 2.7314 - val_accuracy: 0.6787


In [ ]:
# Fine tuning the Feature Extraction Model 
baseModel.trainable = True
for layer in model.layers[1].layers:
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = False
        
model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics = ["accuracy"])

# Train it again 
hist_results_tuned = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=EPOCHS
)
